## Pull from Slack RTM API and write to Kafka

In [7]:
import os
import time
from slackclient import SlackClient

# add your token and start up the SlackClient
# token can be obtained here: https://api.slack.com/docs/oauth-test-tokens
token = 'TOKEN!!!'
sc = SlackClient(token)

# print out all the channels in the given team for the Slack token
channels = []
for channel_dictionary in sc.api_call("channels.list")['channels']:
    channels.append(channel_dictionary)

# for each channel, print the channel id and the channel name
for channel in channels:
    print "Channel name:", channel['name']
    print "Channel id:", channel['id']
    print "\n"

Channel name: architecture
Channel id: C2G7V1D0S


Channel name: bot-sandbox
Channel id: C2CNEMD0S


Channel name: data
Channel id: C2H1W0R8V


Channel name: general
Channel id: C2BSVGJQ4


Channel name: github-repo
Channel id: C2CRHAP61


Channel name: name-selection
Channel id: C2CRJL0MP


Channel name: next-meeting
Channel id: C2DFNQ5PA


Channel name: nlp-methodology
Channel id: C2CV4G41G


Channel name: random
Channel id: C2BSVGJSY


Channel name: tech-stuff
Channel id: C2CUMH2BA




In [8]:
%%writefile pullSlack.py
#!/usr/bin/python

# import the Kafka consumers and producers 
# like Kevan did in his example
import threading, logging, time
from kafka import KafkaConsumer, KafkaProducer

# create a Kafka producer, something that writes to Kafka
class Producer(threading.Thread):
    daemon = True

    # the run function that will actually be the method that 
    # writes to Kafka
    def run(self, topic, writing):
        producer = KafkaProducer(bootstrap_servers='localhost:9092')
        
        # send the topic and writing that was sent
        producer.send(topic, writing)


def main():
            
    # start the producer
    Producer.start()

    # check every second for new messages
    if sc.rtm_connect():
        while True:
            new = sc.rtm_read()
            time.sleep(1)

            # if we actually got a message
            if len(new) > 0:

                # go through each message and 
                # grab the channel and the message
                for message in new:
                    
                    # try to grab the message and the
                    # channel and also try to write
                    # it to Kafka
                    try: 
                        text = message['text']
                        channel = message['channel']
                        Producer.run(channel,text)
                    
                    # we'll pass if that doesn't work
                    # because the information we got was
                    # likely not a message but just a 
                    # status update, e.g. someone going
                    # from away to active status
                    except:
                        pass
main()

Writing pullSlack.py
